In [1]:
#Import required modules

import torch
import torch.nn as nn
import torch.optim as optim
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import sys
import pickle
import re
import logging
import random
import pickle
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from collections import namedtuple
%matplotlib inline

In [2]:
# Logging configuration

# LOG_FORMAT = '%(asctime)s, %(levelname)-8s [%(filename)s:%(funcName)s:%(lineno)d] %(message)s'
LOG_FORMAT = '%(asctime)s, %(levelname)-8s %(message)s'

logger = logging.getLogger('MorphNet-Pytorch')
logging.basicConfig(format=LOG_FORMAT, datefmt='%d-%m-%Y:%H:%M:%S', level=logging.INFO, stream=sys.stdout)

In [3]:
# Select CUDA as device if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info("Using {} as default device".format(device))

21-12-2018:21:11:04, INFO     Using cuda as default device


In [4]:
# Function for padding
def pad_sequences(sequences,  padding_value=0):
    """This function transforms a list of Torch Tensors into a 2D Tensor of shape (num_samples, num_timesteps). 
    num_timesteps is the length of the longest sequence.
    
    Args:
        sequences (list): List of Torch Tensors (´torch.tensor)
        
    Returns:
        ´torch.tensor´: padded tensor

    """
    max_size = sequences[0].size()
    max_len = max([s.size(0) for s in sequences])
    out_dims = (len(sequences), max_len)

    out_tensor = torch.zeros(*out_dims, dtype=torch.long)
    for i, tensor in enumerate(sequences):
        length = tensor.size(0)
        out_tensor[i, :length] = tensor

    return out_tensor

In [5]:
class MorphDisambDataset(Dataset):
    """Morphological disambiguation dataset inherited from ´torch.utils.data.Dataset´
    
    Read from surface words and their analyzes from a file and convert to x->y mapping (surfacewords->analyzes)
    Sentences are considered as batchs
    
    Todo:
        save and load methds implementation
    """
    # Namedtuple o store features of words including surface for, candidate roots and candidate tags
    #     First elements of roots array and tags array are the ones labeled as corrects. 
    WordStruct = namedtuple("WordStruct", ["surface_word", "roots", "tags"])
    
    # Regular expressions to sperate root and tags
    analysis_regex = re.compile(r"^([^\\+]*)\+(.+)$", re.UNICODE)
    tag_seperator_regex = re.compile(r"[\\+\\^]", re.UNICODE)
    split_root_tags_regex = re.compile(r"^([^\\+]+)\+(.+)$", re.IGNORECASE)
    
    def __init__(self, file_path, mode='train', case_sensitive=True, max_sentences=0):
        """Create a MorphDisambDataset reading surface words and their analyzes from a file.
        
        Args:
            file_path (str): Dataset file path.
            mode (str): If 'training', training mode will be activated 
                that means the vocabulary dicts for characters are created and updated.
                If 'testing', the vocab dicts will not change and the data will be encoded according to existing vocabs
            case_sensitive (bool): If False, all characters will be lowercased in the dataset
            max_sentences (int): Maximum nummber of sentences in the sentences. '0' means no limitation.
        
        Example:
            >>> dataset = MorphDisambDataset('dataset.txt', mode='train', max_sentences=50)
        """
        
        
        self.case_sensitive = case_sensitive
        self.ix = -1
        
        # Special characters
        self.EOS_token = '<e>'
        self.PAD_token = '<p>'
        self.START_token = '<s>'
        
        
        self.sentences = []
        sentence = []
        
        if mode == 'train':
            # Two vocab dicts: (1) input characters vocabulay and 
            #   (2) output characters and tags (e.g., Noun, P3sg) vocabulary 
            self.input_char2id = dict()
            self.tag2id = dict()
            
            # Add special characters to vacab dicts
            self.input_char2id[self.PAD_token] = len(self.input_char2id)
            self.tag2id[self.PAD_token] = len(self.tag2id)
            self.input_char2id[self.EOS_token] = len(self.input_char2id)
            self.tag2id[self.EOS_token] = len(self.tag2id)
            self.tag2id[self.START_token] = len(self.tag2id)
        
        # Read from given file
        with open(file_path, 'r', encoding="UTF-8") as f:
            for line in f:
                # skip empty lines
                trimmed_line = line.strip(" \r\n\t")
                if trimmed_line.startswith("<S>") or trimmed_line.startswith("<s>"):
                    sentence = []
                # Sentence completed with sentence end tag. Ad current sentence to sentences list
                elif trimmed_line.startswith("</S>") or trimmed_line.startswith("</s>"):
                    if len(sentence) > 0:
                        self.sentences.append(sentence)
                        # If maximum number of sentences exceeded, stop immediately
                        if len(self.sentences) >= max_sentences > 0:
                            break
                # Skip unnecesarry tags in the file
                elif len(trimmed_line) == 0 or "<DOC>" in trimmed_line or trimmed_line.startswith(
                        "</DOC>") or trimmed_line.startswith(
                        "<TITLE>") or trimmed_line.startswith("</TITLE>"):
                    pass
                # Seperate surface, root and tags
                else:
                    parses = re.split(r"[\t ]", trimmed_line)
                    surface = parses[0]
                    analyzes = parses[1:]
                    roots = [self._get_root(analysis) for analysis in analyzes]
                    tags = [self._get_tags(analysis) for analysis in analyzes]
                    if not self.case_sensitive:
                        surface = self.lower(surface)
                        roots = [self.lower(root) for root in roots]
                    
                    # Create word object and add it to the sentence
                    current_word = self.WordStruct(surface, roots, tags)
                    if mode == 'train':
                        self.update_dicts(surface, roots, tags)
                    sentence.append(current_word)
                    
        print("Dataset created succesfully.")
        
    def update_dicts(self, surface, roots, tags):
        """Update vocab dicts for given surface, roots and tags
        
        Args:
            surface (str): Surface form af the word.
            roots (list): possible roots of the word. (list of strings)
            tags (list): possible analyzes of the word. (list of lists of strings)
        """
        
        # Update vocab for input characters using surface worm of words
        for ch in surface:
            if ch not in self.input_char2id:
                self.input_char2id[ch] = len(self.input_char2id)

        # Update output vocab using roots and tags, 
        #     output vocab (´tag2id´) contains both characters and unique tags (e.g., Noun)
        for root in roots:
            for ch in root:
                if ch not in self.tag2id:
                    self.tag2id[ch] = len(self.tag2id)
        for cur_tags in tags:
            for tag in cur_tags:
                if tag not in self.tag2id:
                    self.tag2id[tag] = len(self.tag2id)
        
    def _get_root(self, analysis):
        """Extract root from given analyis
        
        Args:
            analysis (str): Analysis of a word (e.g., ev+Noun+A3sg+P3sg+Nom)
            
        Returns:
            str: root of the word extracted from analysis
            
        """
        if analysis.startswith("+"):
            return "+"
        else:
            return self.analysis_regex.sub(r"\1", analysis)
        
    def _get_tags(self, analysis):
        """Extract tags from given analyis
        
        Args:
            analysis (str): Analysis of a word (e.g., ev+Noun+A3sg+P3sg+Nom)
            
        Returns:
            list: tags of the word extracted from analysis
            
        """
        
        if analysis.startswith("+"):
            return self.tag_seperator_regex.split(analysis[2:])
        else:
            return self.tag_seperator_regex.split(self.analysis_regex.sub(r"\2", analysis))
        
    def lower(self, text):
        """Lowercase a given text string (focused on Turkish texts)
        
        Args:
            text (str): Any text in Turkish
            
        Returns:
            str: lowercased Turkish text
            
        """
            
        text = text.replace("Ü", "ü")
        text = text.replace("Ğ", "ğ")
        text = text.replace("İ", "i")
        text = text.replace("Ş", "ş")
        text = text.replace("Ç", "ç")
        text = text.replace("Ö", "ö")
        return text.lower()
    
    def _encode(self, tokens, vocab):
        """Encodes given char tokens by converting to indexes.
        
        Args:
            tokens (list): List of tokens
            vocab (dict): Vocab dictionary where keys are tokens and values are integers.
            
        Returns:
            torch.LongTensor: encoded tensor
            
        """
        
        return torch.LongTensor([vocab[i] for i in tokens if i in vocab] + [vocab[self.EOS_token]], device=device)
    
    def _encode_union(self, tokens1, tokens2, vocab):
        """Encodes given 2 collections of char tokens by converting to indexes.
        
        Args:
            tokens1 (list): List of tokens
            tokens2 (list): List of tokens
            vocab (dict): Vocab dictionary where keys are tokens and values are integers.
            
        Returns:
            torch.LongTensor: encoded tensor
            
        """
        
        return torch.LongTensor([vocab[i] for i in tokens1 if i in vocab] 
                                + [vocab[i] for i in tokens2 if i in vocab] + [vocab[self.EOS_token]] , device=device)
        
    def encode_sentence(self, sentence, encode_analyzes=True):
        """Encodes a sentence by encding each words in the sentence.
        
        Args:
            sentence (list): List of words (´MorphDisambDataLoader.WordStruct´)
            
        Returns:
            tuple: encoded sentence tuple consists of 
                encoded surface word (´torch.LongTensor´) and encoded analyzes (´torch.LongTensor´)
            
        """
        
        x = []
        y = []
        
        # Encode surface words according to input vocab dic ´self.input_char2id´
        encoded_surfaces = [self._encode(word.surface_word, self.input_char2id) for word in sentence]
        # Encode analyzes (roots and tags) according to output vocab dic ´self.tag2id
        encoded_analyzes = None
        if encode_analyzes:
            encoded_analyzes = [self._encode_union(word.roots[0], word.tags[0], self.tag2id) for word in sentence]
        
        # Apply padding
        x = pad_sequences(encoded_surfaces)
        y = None
        if encode_analyzes:
            y = pad_sequences(encoded_analyzes)
        
        return x, y
        
    
    def __getitem__(self, ix):
        """Returns encoded sentence in given index
        
        Args:
            ix (int): index of the sentence
            
        Returns:
            tuple: encoded sentence tuple consists of 
                encoded surface word (´torch.LongTensor´) and encoded analyzes (´torch.LongTensor´)
        
        """
        
        sentence = self.sentences[ix]
        return self.encode_sentence(sentence)
    
    def __iter__(self):
        """Restarts iteration"""
        
        self.ix = -1
        return self
    
    def __next__(self):
        """Returns the next sentence in the dataset
        
        Returns:
            tuple: encoded sentence tuple consists of 
                encoded surface word (´torch.LongTensor´) and encoded analyzes (´torch.LongTensor´)
                
        Raises:
            StopIteration: If all sentences are iterated. ´self.__iter__()´ can be called to restart iteration
            
        """
        self.ix += 1
        if self.ix < len(self.sentences):
            sentence = self.sentences[self.ix]
            return self.encode_sentence(sentence)
        else:
            raise StopIteration
        
    def __len__(self):
        """Returns the number of sentences in the dataset
        
        Returns:
            int: the length of the dataset
        """
        
        return len(self.sentences)
    
    def save(self, path="morph_disam_data", save_sentences=False):
        """Save the dataset. The vocabs and the data (optional).
        
        Args:
            save_sentences (bool): If True sentences of the dataset will be saved. Default is False  
        """
        if not os.path.exists(path):
            os.mkdir(path)
            
        with open("{}/char2id".format(path), "wb") as f:
            pickle.dump(self.input_char2id, f)
        with open("{}/tag2id".format(path), "wb") as f:
            pickle.dump(self.tag2id, f)
            
        if save_sentences:
            with open("{}/sentences".format(path), "wb") as f:
                pickle.dump(self.sentences, f)         
        
    def load(self, path="morph_disam_data", load_sentences=False):
        """Load the dataset. The vocabs and the data (optional).
        
        Args:
            load_sentences (bool): If True sentences of the dataset will be loaded. Default is False 
                Be careful, your saved data shoud contain sentences as well. 
        """
        if not os.path.exists(path):
            os.mkdir(path)
            
        with open("{}/char2id".format(path), "rb") as f:
            self.input_char2id = pickle.load(f)
        with open("{}/tag2id".format(path), "rb") as f:
            self.tag2id = pickle.load(f)
            
        if load_sentences:
            with open("{}/sentences".format(path), "rb") as f:
                self.sentences = pickle.load(f)    
        
    
    

In [6]:
class EncoderRNN(nn.Module):
    """A bidirectional GRU as the context encoder.
    
    The inputs are encoded words in a sentence. 
    EncoderRNN firstly apply bidirectional GRU to characters of words to build word representations
    Then apply a second level bidirectional GRU to the word representations.
    By doing so it generates context-aware representations of each word in a sentence which then be used in decoding.
        
    """
    
    def __init__(self, embedding_size, hidden_size1, hidden_size2, vocab_len, dropout_ratio=0.2):
        """ Initialize an EncoderRNN object
        
        Args:
            embedding_size (int): the dimension of the input character embeddings
            hidden_size1 (int): The number of units in first-level gru (char-gru)
            hidden_size2 (int): The number of units in second-level gru (context-gru)
            vocab_len (int): Number of unique characters to initialize character embedding module
            dropout_ratio(float): Dropout ratio, dropout applied to the outputs of both gru and embedding modules
        """
        super(EncoderRNN, self).__init__()
        
        # Hyper-parameters
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.embedding_size = embedding_size
        
        # Initialize modules
        self.embedding = nn.Embedding(vocab_len, embedding_size)
        self.char_gru = nn.GRU(embedding_size, hidden_size1, bidirectional=False, num_layers=1, batch_first=True)
        self.word_gru = nn.GRU(hidden_size1, hidden_size2, bidirectional=True, num_layers=1, batch_first=True)
        self.dropout = nn.Dropout(dropout_ratio)
        
        # Initilize hidden units of grus with Xavier initialization
        self.char_gru_hidden, self.word_gru_hidden = self.init_hidden()

    def init_hidden(self):
        """Initializes the hidden units of each gru
        
        """
        return (
            torch.zeros(1, 1, self.hidden_size1, device=device),
            torch.zeros(2, 1, self.hidden_size2, device=device)
        )
    
    
    def forward(self, x):
        """Forward pass of EncoderRNN
        
        Embedding layer, first level grus and second level grus are applied to input  tensor
        Dropouts are applied between all layers with parameters
        
        Returns:
            torch.Tensor: word embeddings generated by first-level character grus
            torch.Tensor: context-aware representations of the words occur in the sentence
        
        """
        # Batch size should be 1, sentences are batche in our implementation
        assert x.size(0) == 1, "Batch size should be 1 since each sentence is considered as a mini-batch"
        
        self.char_gru_hidden, self.word_gru_hidden = self.init_hidden()
        
        # Embedding layer
        embeddeds = self.embedding(x)
        embeddeds = self.dropout(embeddeds)
        
        # First-level gru layer (char-gru to generate word embeddings)
        word_embeddings_list = []
        for i, char_embeddings in enumerate(embeddeds[0]):
            char_embeddings = char_embeddings.view(1, *char_embeddings.shape)
            outputs, _ = self.char_gru(char_embeddings, self.char_gru_hidden)
            word_embeddings_list.append(outputs[0][-1].view(1, 1, -1))
        word_embeddings = torch.cat(word_embeddings_list, 1)
            
        # Second-level gru ayer (context-gru)
        context_embeddings = self.word_gru(word_embeddings, self.word_gru_hidden)[0]
        return word_embeddings, context_embeddings
        


In [7]:
class DecoderRNN(nn.Module):
    """ The module generates characters and tags sequentially to construct a morphological analysis
        
    Inputs a context representation of a word and apply grus 
    to predict the characters in the root form and the tags in the analysis respectively
    
    """
    def __init__(self, embedding_size, hidden_size, vocab, dropout_ratio=0):
        """Initialize the decoder object
        
        Args:
            embedding_size (int): The dimension of embeddings 
                (output embeddings includes character for roots and tags for analyzes)
            hidden_size (int): The number of units in gru
            vocab (dict): Vocab dictionary where keys are either characters or tags and the values are integer
            dropout_ratio(float): Dropout ratio, dropout applied to the outputs of both gru and embedding modules
        """
        super(DecoderRNN, self).__init__()
        
        # Hyper parameters
        self.hidden_size = hidden_size
        
        # Vocab and inverse vocab to converts output indexes to characters and tags
        self.vocab = vocab
        self.index2token = {v: k for k, v in vocab.items()}
        self.vocab_size = len(vocab)
        
        # Layers
        self.W = nn.Linear(2 * hidden_size, hidden_size)
        self.embedding = nn.Embedding(len(vocab), embedding_size)
        self.gru = nn.GRU(embedding_size, hidden_size, 2)
        self.classifier = nn.Linear(hidden_size, len(vocab))
        self.dropout = nn.Dropout(p=dropout_ratio)
        self.softmax = nn.LogSoftmax(dim=1)
        self.relu = nn.ReLU()
        
    def forward(self, word_embedding, context_vector, y, target_length, teacher_forcing_ratio=1.0):
        """Forward pass of DecoderRNN
        
        Inputs a context-aware vector of a word and produces an analysis consists of root+tags
        
        Args:
            context_vector (´torch.tensor´): Context-aware representation of a word
            target_length (int): The length of the correct analysis
            teacher_forcing_ratio: probability to use teacher forcing
            
        Returns: 
            ´torch.tensor´: scores in each time step
        """
        # Decide whether to use teacher forcing
        if random.random() < teacher_forcing_ratio:
            use_teacher_forcing = True
        else:
            use_teacher_forcing = False
            
        # Initilize gru hidden units with context vector (encoder output)
        context_vector = context_vector.view(1, *context_vector.size())
        context_vector = self.relu(self.W(context_vector).view(1, 1, self.hidden_size))
        word_embedding = word_embedding.view(1, 1, self.hidden_size)
        hidden = torch.cat([context_vector, word_embedding], 0)
        
        # Oupput shape (maximum length of a an analyzer, output vocab size)
        scores = torch.zeros(target_length, self.vocab_size).to(device)
        
        # First predicted token is sentence start tag: 2
        predicted_token = torch.LongTensor(1).fill_(2).to(device)
        
        # Generate char or tag sequentially
        for di in range(target_length):
            # Propagate layers
            embedded = self.embedding(predicted_token).view(1, 1, -1)
            embedded = torch.relu(embedded)
            embedded = self.dropout(embedded)
            
            output, hidden = self.gru(embedded, hidden)
            output = self.dropout(output)
            output = self.classifier(output[0])
            
            # Save scores in each time step
            scores[di] = self.softmax(output)
            
            if use_teacher_forcing:
                # Use actual tag as input (teacher forcing)
                predicted_token = y[di]
            else:
                # Save the predicted token index which embedding layer inputs
                topv, topi = output.topk(1)
                predicted_token = topi.squeeze().detach()
            
        # Return the scores for each time step
        return scores
    
    def predict(self, word_embedding, context_vector, max_len=50):
        """Forward pass of DecoderRNN for prediction only
        
        The loop for gru is stopped as soon as the end of sentence tag is produced twice.
        The first end of sentence tag indicates the end of the root while the second one indicates the end of tags 
        
        Args:
            context_vector (´torch.tensor´): Context-aware representation of a word
            max_len (int): Maximum length of produced analysis (Defaault: 50)
            
        Returns:
            tuple: (scores:´torch.tensor´, predictions:list)
        
        """
        
        # Initilize gru hidden units with context vector (encoder output)
        context_vector = context_vector.view(1, *context_vector.size())
        context_vector = self.relu(self.W(context_vector).view(1, 1, self.hidden_size))
        word_embedding = word_embedding.view(1, 1, self.hidden_size)
        hidden = torch.cat([context_vector, word_embedding], 0)
        
        # Oupput shape (maximum length of a an analyzer, output vocab size)
        scores = torch.zeros(max_len, self.vocab_size).to(device)
        
        # First predicted token is sentence start tag: 2
        predicted_token = torch.LongTensor(1).fill_(2).to(device)
        
        # Generate char or tag sequentially
        eos_count = 0
        predictions = []
        for di in range(max_len):
            embedded = self.embedding(predicted_token).view(1, 1, -1)
            embedded = torch.relu(embedded)
            output, hidden = self.gru(embedded, hidden)
            output = self.classifier(output[0])
            scores[di] = self.softmax(output)
            topv, topi = output.topk(1)
            predicted_token = topi.squeeze().detach()
            # Increase eos count if produced output is eos
            if predicted_token.item() == 1:
                eos_count += 1
            # Add predicted output to predictions if it is not a special character such as eos or padding
            if predicted_token.item() > 2:
                predictions.append(self.index2token[predicted_token.item()])
            # If eos count greater than 1, stop iteration
            if eos_count >= 2:
                break

        return scores, predictions

In [57]:
def _convert_to_morph_analyzer_form(sequence_arr):
    """Convert a sequence of decoder outputs to standard morphological analyzer form
    
    Arguments:
        sequence_arr (list): list of labels generated by a ´DecoderRNN´
        
    Returns:
        str: morphological analysis in standard form such as ev+Noun+A3sg+Pnon+Nom
    """
    
    res = []
    for x in sequence_arr:
        if len(x) == 1:
            res.append(x)
        else:
            res.append("+")
            res.append(x)
    res = "".join(res)
    res = res.replace("+DB", "^DB")
    return res
    
def analyze(sentence, enc, dec, dataset, device=torch.device("cpu"), max_len=50):
    """ Generates analyzes of given sentence
    
    Args:
        sentence (list of strings or str or list of namedtuple): List of tokens or string. 
            If it is type of string, sentence will be split into tokens by whitespace tokenizer
            If it is list of namedtuples (WordStruct), no preprocessing is needed
        enc (EncoderRNN): encoder
        dataset (´MorphDisambDataset´): 
        dec (DecoderRNN): decoder
            
    Returns:
        list: analyzes of each token in the sentence
    """
    
    if len(sentence) == 0:
        return []
    
    if isinstance(sentence, tuple):
        if isinstance(sentence[0], torch.Tensor):
            encoded_surface = sentence[0]
        assert isinstance(sentence[0], torch.Tensor), "Sentence should contains tuples of `torch.Tensors`"
        encoded_surface = sentence[0]
    else:
        if isinstance(sentence, list) and isinstance(sentence[0], tuple):
            # Pass if sentence is already in WordStruct format
            words = sentence
        else:
            # Split sentence if it is string
            if isinstance(sentence, str):
                sentence = sentence.split(" ")

            # Convert words to WordStruct format
            words = []
            for token in sentence:
                words.append(dataset.WordStruct(token, [], []))
    
        # Encoding, convert to torch tensors
        encoded_surface, _ = dataset.encode_sentence(words, encode_analyzes=False)
        encoded_surface = encoded_surface.to(device)
    
    assert encoded_surface is not None, "Bad input"
    encoded_surface = encoded_surface.to(device)
    # Run encoder
    word_embeddings, context_embeddings = enc(encoded_surface.view(1, *encoded_surface.size()))
    # Run decoder for each word
    analyzes = []
    word_count = context_embeddings.size(1)
    for word_ix in range(word_count):
        scores, word_analysis = dec.predict(word_embeddings[0,word_ix], context_embeddings[0,word_ix], max_len=max_len)
        analyzes.append(_convert_to_morph_analyzer_form(word_analysis))
                
    return analyzes

In [78]:
def calc_acc_sentence(sentence, enc, dec, dataset, device=torch.device("cpu")):
    """Accuracy calcuator for given encoder and decoder models over given sentence
    
    Arguments:
        sentence (list): List of words (´collections.namedtuple´)
        enc (EncoderRNN): encoder
        dec (DecoderRNN): decoder
        dataset (´MorphDisambDataset´): Morphological Disambiguation Dataset (Required for encoding words and tags)
        device (´torch.device´): indicate which device will be used (cpu or gpu)
        
    Retruns:
        int: word count
        int: correcst count
    """
    
    corrects = 0
    total = 0
    
    # Analyze words
    analyzes = analyze(sentence, enc, dec, dataset, device=device)
    
    for word, analysis in zip(sentence, analyzes):
        if analysis == word.roots[0] + '+' +  '+'.join(word.tags[0]).replace('+DB', '^DB'):
            corrects += 1
        else:
            print("{} <=> {}".format(analysis, word.roots[0] + '+' +  '+'.join(word.tags[0]).replace('+DB', '^DB')))
        total += 1
    
    return total, corrects


def calc_acc(test_data_path, enc, dec, dataset, device=torch.device("cpu")):
    """Accuracy calcuator for given encoder and decoder models over given dataset
    
    Arguments:
        test_data_path (str): The path of the test data
        enc (EncoderRNN): encoder
        dec (DecoderRNN): decoder
        dataset (´MorphDisambDataset´): Morphological Disambiguation Dataset (Required for encoding words and tags)
        device (´torch.device´): indicate which device will be used (cpu or gpu)
            
    Returns:
        accuracy (float): overall accuracy
        amb_accuracy (float): ambiguous accuracy
    """
    total = 0
    corrects = 0
    # Read from given file
    sentence = []
    with open(test_data_path, 'r', encoding="UTF-8") as f:
        for line in f:
            # skip empty lines
            trimmed_line = line.strip(" \r\n\t")
            if trimmed_line.startswith("<S>") or trimmed_line.startswith("<s>"):
                sentence = []
            # Sentence completed with sentence end tag. Ad current sentence to sentences list
            elif trimmed_line.startswith("</S>") or trimmed_line.startswith("</s>"):
                if len(sentence) > 0:
                    t, c = calc_acc_sentence(sentence, enc, dec, dataset, device=device)
                    total += t
                    corrects += c
            # Skip unnecesarry tags in the file
            elif len(trimmed_line) == 0 or "<DOC>" in trimmed_line or trimmed_line.startswith(
                    "</DOC>") or trimmed_line.startswith(
                    "<TITLE>") or trimmed_line.startswith("</TITLE>"):
                pass
            # Seperate surface, root and tags
            else:
                parses = re.split(r"[\t ]", trimmed_line)
                surface = parses[0]
                analyzes = parses[1:]
                roots = [dataset._get_root(analysis) for analysis in analyzes]
                tags = [dataset._get_tags(analysis) for analysis in analyzes]
                if not dataset.case_sensitive:
                    surface = dataset.lower(surface)
                    roots = [dataset.lower(root) for root in roots]

                # Create word object and add it to the sentence
                current_word = dataset.WordStruct(surface, roots, tags)
                sentence.append(current_word)
                
    return corrects * 1.0 / total

In [10]:
# Max sentence length
max_words = 50

# SENTENCES ARE BATCHES, DO NOT CHANGE
BATCH_SIZE = 1 

# Number of epochs
num_epochs = 500

# Encoder hyper-parmeters
embedding_size = 64
char_gru_hidden_size = 512
word_gru_hidden_size = 512
encoder_lr = 0.01
encoder_dropout = 0.3
encoder_weight_decay = 0.01
encoder_scheduler_factor = 0.9
encoder_momentum = 0.1

# Decoder hyper-parmeters
output_embedding_size = 256
decoder_gru_hidden_size = 512
decoder_lr = 0.01
decoder_dropout = 0.3
decoder_weight_decay = 0.01
decoder_scheduler_factor = 0.9
decoder_momentum = 0.1

In [11]:
train_dataset = MorphDisambDataset(file_path='data/data.train.txt', mode='train', max_sentences=0)
eval_dataset = MorphDisambDataset(file_path='data/test.merge', mode='test', max_sentences=0)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=10)
eval_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=10)

Dataset created succesfully.
Dataset created succesfully.


In [12]:
# Initialize encoder and decoder
encoder = EncoderRNN(embedding_size, char_gru_hidden_size, word_gru_hidden_size, 
                     len(train_dataset.input_char2id), dropout_ratio=encoder_dropout)
encoder = encoder.to(device)

decoder = DecoderRNN(output_embedding_size, word_gru_hidden_size, train_dataset.tag2id, dropout_ratio=decoder_dropout)
decoder = decoder.to(device)

# Define loss and optimizers
criterion = nn.NLLLoss(ignore_index=0).to(device)


In [13]:
# Learning rate decay
def lr_decay_step(optimizer, lr, factor=0.1, weight_decay=0.0):
    """Learning rate decay step
    
    Args:
        optimizer:
        lr:
        factor:
    Returns:
        int: lr 
        optimizer
    """
    lr = factor * lr
    optimizer = optim.SGD(encoder.parameters(), 
                          lr=lr
                         )
    return lr, optimizer

In [151]:
encoder_lr=0.000005
decoder_lr=0.000005
encoder_optimizer = optim.Adam(encoder.parameters(), 
                              lr=encoder_lr
                             )
decoder_optimizer = optim.Adam(decoder.parameters(), 
                              lr=decoder_lr
                             )

In [156]:
# Let the training begin
notify_each = 500
update_lr = 10000
for epoch in range(num_epochs):
    
    total_train_loss = 0.0
    total_val_loss = 0.0
    previous_loss = 0.0
    # Training part
    encoder.train()
    decoder.train()
        
    for ix, (x, y) in enumerate(train_loader):
        # Skip sentences longer than max_words
        if x.size(1) > max_words:
            continue

        # Clear gradients for each sentence
        encoder.zero_grad()
        decoder.zero_grad()

        # Send input to the device
        x = x.to(device)
        y = y.to(device)
        
        # Init loss value
        loss = 0
        
        # Run encoder
        word_embeddings, context_embeddings = encoder(x)
        
        # Run decoder for each word
        word_count = context_embeddings.size(1)
        sentence_loss = 0.0
        for word_ix in range(word_count):
            target_length = y[0][word_ix].size(0)
            decoder_outputs = decoder(word_embeddings[0, word_ix], context_embeddings[0, word_ix], 
                                      y[0][word_ix], target_length=target_length)
            loss += criterion(decoder_outputs, y[0][word_ix])
            sentence_loss += loss.item()  / y[0][word_ix].size(0)
        sentence_loss /= word_count
        total_train_loss += sentence_loss

        # Optimization
        loss.backward()
                
        encoder_optimizer.step()
        decoder_optimizer.step()
        
        if (ix+1) % notify_each == 0:
            print("Epoch {}. Sample: {}. Train Loss: {}, Encoder lr: {}, Decoder lr: {}".format(
                   epoch, (ix+1), total_train_loss / (ix+1), encoder_lr, decoder_lr)
                 )
            print(analyze(train_dataset[0], encoder, decoder, train_dataset, device=device))
        
    # Validation Accuracy
    with torch.no_grad():
        encoder.eval()
        decoder.eval()
        # Calculate losses and notify
        train_loss = total_train_loss / len(train_dataset)
        val_acc = calc_acc('data/test.merge', encoder, decoder, train_dataset, device)
        
    print("Epoch {} is completed. Train Loss: {}, Val Acc: {}, Encoder lr: {}, Decoder lr: {}".format(
        epoch, train_loss, val_acc, encoder_lr, decoder_lr)
         )
    print(analyze(train_dataset[0], encoder, decoder, train_dataset, device=device))
    
    

Epoch 0. Sample: 500. Train Loss: 0.0030959652130395946, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
Epoch 0. Sample: 1000. Train Loss: 0.0031227747918675997, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
Epoch 0. Sample: 1500. Train Loss: 0.003015095710028964, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
Epoch 0. Sample: 2000. Train Loss: 0.002889586397471966, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
Epoch 0

Epoch 0. Sample: 17000. Train Loss: 0.002775393159249289, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
Epoch 0. Sample: 17500. Train Loss: 0.002796955235322509, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
Epoch 0. Sample: 18000. Train Loss: 0.0028066808061215026, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
Epoch 0. Sample: 18500. Train Loss: 0.002800143787447414, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
Epo

Epoch 0. Sample: 34000. Train Loss: 0.0028381788399098216, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
Epoch 0. Sample: 34500. Train Loss: 0.002844503880288555, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
Epoch 0. Sample: 35000. Train Loss: 0.0028449695536212623, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
Epoch 0. Sample: 35500. Train Loss: 0.0028498970972950953, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
E

ara+Noun+A3pl+P3sg+Loc <=> ara+Noun+A3pl+P3pl+Loc
mitsubiyim+Noun+A3sg+Pnon+Nom <=> Mitsubishi+Noun+Prop+A3sg+Pnon+Nom
*UNKNOWN*+Verb+Pos^DB+Noun+Inf2+A3sg+Pnon+Dat <=> '+Punc
çok+Det <=> çok+Adj
ak+Verb+Pos+Fut+A1sg <=> ak+Verb+Pos^DB+Noun+FutPart+A3sg+P3sg+Nom
fiyat+Noun+A3pl+P3pl+Nom <=> fiyat+Noun+A3pl+P3sg+Nom
art+Verb+Pos^DB+Adj+PastPart+P3sg <=> art+Verb+Pos^DB+Noun+PastPart+A3sg+P3sg+Nom
bulun+Verb+Pos^DB+Adj+PastPart+P3sg <=> bulun+Verb+Pos^DB+Noun+PastPart+A3sg+P3sg+Nom
Kamu+Noun+Prop+A3sg+Pnon+Nom <=> kamu+Noun+A3sg+Pnon+Nom
eski+Adj+Noun+A3sg+Pnon+Nom <=> eski+Adj
il+Noun+A3sg+Pnon+Dat <=> ile+Postp+PCNom
ile+Postp+PCNom <=> ile+Conj
aç+Verb+Pos^DB+Noun+PastPart+A3sg+P3sg+Nom <=> aç+Verb+Pos^DB+Adj+PastPart+P3sg
Rp+Noun+Prop+A3sg+Pnon+Nom <=> rp+Noun+A3sg+Pnon+Nom
Yök+Noun+Prop+A3sg+Pnon+Nom <=> yök+Noun+A3sg+Pnon+Nom
kemal+Noun+A3sg+Pnon+Nom <=> Kemal+Noun+Prop+A3sg+Pnon+Nom
uygula+Verb+Pos^DB+Adj+PastPart+P3pl <=> uygula+Verb+Pos^DB+Noun+PastPart+A3sg+P3pl+Nom
Odtü+Noun+P

Epoch 1. Sample: 13500. Train Loss: 0.0027328642635049017, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
Epoch 1. Sample: 14000. Train Loss: 0.0027252708170748367, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
Epoch 1. Sample: 14500. Train Loss: 0.002721786968825055, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
Epoch 1. Sample: 15000. Train Loss: 0.002733357246005843, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
Ep

Epoch 1. Sample: 31000. Train Loss: 0.0027547782453354896, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
Epoch 1. Sample: 31500. Train Loss: 0.0027477907035720183, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
Epoch 1. Sample: 32000. Train Loss: 0.002750568811506969, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
Epoch 1. Sample: 32500. Train Loss: 0.0027592879773888856, Encoder lr: 5e-06, Decoder lr: 5e-06
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'da+Conj', 'Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'için+Postp+PCNom', 'görüş+Noun+A3sg+Pnon+Nom', 'iste+Verb+Pos+Past+A3sg']
E

dolar+Noun+A3sg+Pnon+Acc <=> dolar+Noun+A3sg+P3sg+Nom
dolar+Noun+A3sg+Pnon+Acc <=> dolar+Noun+A3sg+P3sg+Nom
gerçek+Noun+A3sg+Pnon+Nom^DB+Verb+Become^DB+Verb+Caus^DB+Verb+Pass+Pos^DB+Adj+PresPart <=> gerçek+Adj^DB+Verb+Become^DB+Verb+Caus^DB+Verb+Pass+Pos^DB+Adj+PresPart
dış+Noun+A3sg+Pnon+Nom <=> dış+Adj
413.39+Num+Real <=> 413.93+Num+Real
ırak+Adj <=> Irak+Noun+Prop+A3sg+Pnon+Nom
Emiroğlu+Noun+Prop+A3sg+Pnon+Nom <=> emiroğlu+Noun+A3sg+Pnon+Nom
Bm+Noun+Prop+A3sg+Pnon+Nom <=> bm+Noun+A3sg+Pnon+Nom
kerkük+Noun+A3sg+Pnon+Nom <=> Kerkük+Noun+Prop+A3sg+Pnon+Nom
yumurtalık+Noun+A3sg+Pnon+Nom <=> yumurtalık+Noun+Prop+A3sg+Pnon+Nom
Bm+Noun+Prop+A3sg+Pnon+Nom <=> bm+Noun+A3sg+Pnon+Nom
karar+Noun+A3sg+Pnon+Acc <=> karar+Noun+A3sg+P3sg+Nom
bağdat+Noun+A3sg+Pnon+Nom <=> Bağdat+Noun+Prop+A3sg+Pnon+Nom
değer+Noun+A3sg+P2sg+Loc <=> değer+Noun+A3sg+P3sg+Loc
çık+Verb+Pos+Fut+A3sg <=> çık+Verb+Pos^DB+Adj+FutPart+Pnon
karşılaş+Verb+Pos+Fut+A1pl <=> karşılaş+Verb+Pos^DB+Adj+FutPart+P3sg
Bm+Noun+Prop+A3sg+

Process Process-381:
Process Process-390:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/eray/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/eray/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/eray/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/eray/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process Process-386:
Process Process-383:
  File "/home/eray/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
Process Process-385:
Process Process-387:
Process Process-389:
Process Process-388:
Process Process-382:
  File "/home/eray/anaconda3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader

  File "/home/eray/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/eray/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/eray/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
KeyboardInterrupt
  File "/home/eray/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/eray/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
  File "/home/eray/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/home/eray/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/eray/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379,

KeyboardInterrupt: 

In [158]:
torch.manual_seed(42)
analyze("hesabıma", encoder, decoder, train_dataset, device=device)

['hesak+Noun+A3sg+P1sg+Dat']

In [157]:
torch.save(encoder.state_dict(), "morph_disam_data/encoder.model")
torch.save(decoder.state_dict(), "morph_disam_data/decoder.model")
train_dataset.save()

In [15]:
encoder.load_state_dict(torch.load("morph_disam_data/encoder.model"))
decoder.load_state_dict(torch.load("morph_disam_data/decoder.model"))
train_dataset.load()

In [155]:
calc_acc('data/data.test.txt', encoder, decoder, train_dataset, device)

uç+Verb+Pos^DB+Adverb+AfterDoingSo+Noun+A3sg+Pnon+Dat <=> uç+Verb+Pos^DB+Adverb+AfterDoingSo
artıkla+Verb+Pos+Aor+A3sg <=> artık+Noun+A3pl+P3sg+Nom
küflecenik+Noun+A3sg+Pnon+Nom+Verb+Pos+Imp+A2sg <=> küf+Noun+A3sg+Pnon+Nom^DB+Verb+Acquire+Pos+Fut+A1pl
değiş+Verb^DB+Verb+Caus+Neg+Fut+A3sg <=> değiş+Verb^DB+Verb+Caus+Neg^DB+Adj+FutPart+Pnon
çelik+Noun+A3sg+Pnon+Nom <=> çelik+Adj
ordu+Noun+A3pl+P3pl+Nom <=> ordu+Noun+A3pl+P3sg+Nom
ev+Noun+A3pl+P3sg+Nom <=> ev+Noun+A3pl+Pnon+Acc
sarar+Noun+A3sg+Pnon+Nom^DB+Verb+Zero+Narr+A3sg <=> sar+Verb+Pos+Aor+Narr+A3sg
hayali+Adj^DB+Noun+Zero+A3sg+P1pl+Abl <=> hayal+Noun+A3sg+P1pl+Abl
örnek+Noun+A3sg+Pnon+Gen <=> örnek+Noun+A3sg+P2sg+Nom
atmosfer+Noun+A3sg+Pnon+Acc <=> atmosfer+Noun+A3sg+P3sg+Nom
koku+Noun+A3pl+P3sg+Nom <=> koku+Noun+A3pl+Pnon+Acc
zorla+Verb^DB+Verb+Pass+Pos^DB+Noun+FutPart+A3sg+P3sg+Nom <=> zorla+Verb^DB+Verb+Pass+Pos+Fut+A1pl
silivet+Noun+A3sg+P3sg+Acc <=> silüetini+Noun+A3sg+Pnon+Nom
tam+Adverb <=> tam+Adj
büyük+Adj+Noun+A3sg+Pnon+N

futbol+Noun+A3sg+Pnon+Nom^DB+Verb+Acquire^DB+Verb+Caus+Pos^DB+Adverb+AfterDoingSo <=> futbolsever+Noun+A3pl+Pnon+Gen
barca+Noun+A3sg+Pnon+Nom <=> bar+Noun+A3sg+Pnon+Equ
900+Num+Card^DB+Noun+Zero+A3sg+Pnon+Acc <=> 900+Num+Card^DB+Noun+Zero+A3sg+P3sg+Nom
wall+Noun+A3sg+Pnon+Nom <=> Wall+Noun+Prop+A3sg+Pnon+Nom
street+Noun+A3sg+Pnon+Nom <=> Street+Noun+Prop+A3sg+Pnon+Nom
velhasul+Noun+A3sg+Pnon+Nom <=> velhasıl+Noun+A3sg+Pnon+Nom
şirketle+Verb+Pos^DB+Noun+Inf2+A3sg+Pnon+Gen <=> şirket+Noun+A3sg+Pnon+Nom^DB+Verb+Become+Pos^DB+Noun+Inf2+A3sg+Pnon+Gen
bırak+Verb+Pos+Imp+A2sg <=> Bırak+Noun+Prop+A3sg+Pnon+Nom
incele+Verb+Pos^DB+Noun+PastPart+A3sg+P3sg+Nom <=> incel+Verb+Pos^DB+Adj+PastPart+P3sg
laftır+Noun+A3sg+Pnon+Nom <=> laf+Noun+A3sg+Pnon+Nom^DB+Verb+Zero+Pres+Cop+A3sg
eski+Adj+Noun+A3sg+Pnon+Nom <=> eski+Adj
laf+Noun+A3sg+Pnon+Acc <=> laf+Noun+A3sg+P3sg+Nom
kükrüyor+Noun+A3sg+Pnon+Nom <=> kükre+Verb+Pos+Prog1+A3sg
git+Verb^DB+Verb+Pass+Pos^DB+Noun+FutPart+A3sg+Pnon+Loc <=> git+Verb^DB+Ve

halıcılık+Noun+A3sg+Pnon+Nom <=> halıcı+Noun+A3sg+Pnon+Nom^DB+Noun+Ness+A3sg+Pnon+Nom
kargo'nun+Noun+A3sg+Pnon+Nom <=> Kargo+Noun+Prop+A3sg+Pnon+Gen
kamyon+Noun+A3sg+P3pl+Abl <=> kamyon+Noun+A3pl+P3sg+Abl
Topçu+Noun+Prop+A3sg+Pnon+Gen <=> Top+Noun+Prop+A3sg+Pnon+Nom^DB+Noun+Agt+A3sg+Pnon+Gen
taşdirek+Noun+A3sg+Pnon+Nom <=> Taşdirek+Noun+Prop+A3sg+Pnon+Nom
kargo'nun+Noun+A3sg+Pnon+Nom <=> Kargo+Noun+Prop+A3sg+Pnon+Gen
bir+Det+Det <=> bir+Det
çember+Noun+A3sg+Pnon+Nom^DB+Adj+With^DB+Verb+Zero+Past+A3sg <=> Çemberlitaş+Noun+Prop+A3sg+Pnon+Nom
cıvar+Noun+A3sg+P2sg+Loc^DB+Adj+Rel <=> cıvarındak+Noun+A3sg+P3sg+Nom
yok+Adj^DB+Verb+Zero+Pres+A2sg <=> yokuş+Noun+A3sg+Pnon+Nom
koca+Verb+Neg+Imp+A2pl <=> kocaman+Adj
taksolar+Noun+A3sg+Pnon+Ins <=> takoz+Noun+A3pl+Pnon+Ins
çakıl+Noun+A3sg+P3sg+Nom <=> çakıl+Noun+A3sg+Pnon+Acc
söz+Noun+A3pl+P3pl+Acc <=> söz+Noun+A3pl+P3sg+Acc
vaad+Noun+A3pl+P3pl+Acc <=> vaadlerini+Noun+A3sg+Pnon+Nom
bu+Det <=> bu+Pron+Demons+A3sg+Pnon+Nom
et+Verb^DB+Verb+Pass+Pos+F

Yılbaş+Noun+Prop+A3sg+Pnon+Acc <=> yılbaş+Noun+A3sg+P3sg+Nom
Yazıcı+Noun+Prop+A3sg+Pnon+Nom <=> yazı+Noun+A3sg+Pnon+Nom^DB+Noun+Agt+A3sg+Pnon+Nom
taş+Verb+Pos^DB+Adj+PresPart^DB+Noun+Zero+A3pl+Pnon+Nom <=> Taşanlar+Noun+Prop+A3sg+Pnon+Nom
24.00+Noun+Time+A3sg+Pnon+Acc <=> 24.00+Noun+Time+A3sg+P3sg+Nom
göynük+Noun+A3sg+Pnon+Nom <=> göynük+Adj
royal+Noun+A3sg+Pnon+Nom <=> Royal+Noun+Prop+A3sg+Pnon+Nom
yarış+Verb+Pos^DB+Adj+PastPart+P3sg <=> yarış+Verb+Pos^DB+Noun+PastPart+A3sg+P3sg+Nom
örgülü+Adj <=> örgü+Noun+A3sg+Pnon+Nom^DB+Adj+With
et+Verb+Pos^DB+Noun+Inf3+A3sg+Pnon+Ins+Adj <=> etek+Noun+A3sg+P3sg+Ins
hayri+Noun+A3sg+Pnon+Nom <=> Hayri+Noun+Prop+A3sg+Pnon+Nom
Gazino+Noun+Prop+A3sg+P3sg+Loc+Adj <=> Gazino+Noun+Prop+A3sg+P3sg+Loc
al+Verb+Pos^DB+Adj+PresPart <=> alan+Noun+A3sg+Pnon+Nom
topla+Verb+Pos+Aor^DB+Adj+Zero <=> top+Noun+A3pl+Pnon+Nom
top+Noun+A3pl+Pnon+Acc <=> top+Noun+A3pl+P3sg+Nom
Ünlü+Noun+Prop+A3sg+Pnon+Nom <=> ünlü+Noun+A3sg+Pnon+Nom
kız+Noun+A3sg+P3sg+Nom <=> kız+Noun+A3s

kılıkçaya+Noun+A3sg+Pnon+Nom <=> Kılıçkaya+Noun+Prop+A3sg+Pnon+Nom
çam+Noun+A3sg+Pnon+Nom^DB+Adj+With^DB+Noun+Zero+A3sg+Pnon+Dat <=> çamlıgöze+Noun+A3sg+Pnon+Nom
Gazibe+Noun+Prop+A3sg+Pnon+Dat <=> gazibey'e+Noun+A3sg+Pnon+Nom
suşehir+Noun+A3sg+P3sg+Nom <=> Suşehri+Noun+Prop+A3sg+Pnon+Nom
proje+Noun+A3sg+P3sg+Dat <=> Proje+Noun+Prop+A3sg+P3sg+Dat
aeg'yi+Noun+A3sg+Pnon+Nom <=> Aeg+Noun+Prop+A3sg+Pnon+Acc
electrolo+Noun+A3sg+Pnon+Nom <=> electrolux+Noun+A3sg+Pnon+Nom
aeg'nin+Noun+A3sg+Pnon+Nom <=> Aeg+Noun+Prop+A3sg+Pnon+Gen
bulun+Verb+Pos^DB+Adj+PastPart+P3sg <=> bulun+Verb+Pos^DB+Noun+PastPart+A3sg+P3sg+Nom
marka+Noun+A3pl+P3pl+Acc <=> marka+Noun+A3pl+P3sg+Acc
Peg+Noun+Prop+A3sg+Pnon+Nom <=> peg+Noun+A3sg+Pnon+Nom
Jak+Noun+Prop+A3sg+Pnon+Nom <=> jak+Noun+A3sg+Pnon+Nom
Kamhiy+Noun+Prop+A3sg+Pnon+Dat <=> kamhi'ye+Noun+A3sg+Pnon+Nom
profilo'yu+Noun+A3sg+Pnon+Nom <=> Profilo+Noun+Prop+A3sg+Pnon+Acc
siemens+Noun+A3sg+Pnon+Nom <=> Siemens+Noun+Prop+A3sg+Pnon+Nom
ayır+Verb^DB+Verb+Pass+Neg^DB+

...

zam+Noun+A3sg+Pnon+Acc <=> zam+Noun+A3sg+P3sg+Nom
al+Verb+Pos^DB+Noun+FutPart+A3sg+P3sg+Acc <=> alacak+Noun+A3sg+P3sg+Acc
sağlam+Adj <=> Sağlam+Noun+Prop+A3sg+Pnon+Nom
çalış+Verb+Pos^DB+Noun+Inf2+A3pl+P3sg+Nom <=> çalış+Verb+Pos^DB+Noun+Inf2+A3pl+Pnon+Acc
genelde+Adverb+Noun+A3sg+Pnon+Nom <=> genelde+Adverb
yap+Verb^DB+Verb+Pass+Pos+Fut+A3sg <=> yap+Verb^DB+Verb+Pass+Pos^DB+Adj+FutPart+Pnon
götür+Verb+Pos^DB+Noun+PastPart+A3pl+P3pl+Acc <=> götür+Verb+Pos^DB+Noun+PastPart+A3sg+P3pl+Acc
ne+Adj <=> ne+Pron+Ques+A3sg+Pnon+Nom
gör+Verb+Pos^DB+Adj+FutPart+P3sg <=> gör+Verb+Pos^DB+Noun+FutPart+A3sg+P3sg+Nom
denge+Noun+A3pl+Pnon+Acc <=> denge+Noun+A3pl+P3sg+Nom
ne+Adverb <=> ne+Adj
çıkar+Noun+A3sg+Pnon+Nom <=> çık+Verb+Pos+Aor+A3sg
imkan+Noun+A3pl+P3sg+Nom <=> imkan+Noun+A3pl+Pnon+Acc
gözük+Verb+Pos^DB+Noun+Inf2+A3pl+P3sg+Nom <=> gözük+Verb+Pos+Neces+Cop+A3sg
ekonomik+Adj+Noun+A3sg+Pnon+Nom <=> ekonomik+Adj
otur+Verb+Pos^DB+Adverb+AfterDoingSo+Noun+A3sg+Pnon+Nom <=> otur+Verb+Pos^DB+Adverb+Aft

anababa+Noun+A3pl+P3sg+Nom <=> anababa+Noun+A3pl+Pnon+Acc
dava+Noun+A3pl+P3pl+Nom <=> dava+Noun+A3pl+P3sg+Nom
saf+Noun+A3sg+Pnon+Nom <=> saf+Adj
istek+Noun+A3pl+P2sg+Nom <=> istek+Noun+A3pl+Pnon+Gen
nasıl+Adj <=> nasıl+Adverb
temellen+Verb+Pos^DB+Noun+PastPart+A3sg+P3pl+Acc <=> temel+Noun+A3sg+Pnon+Nom^DB+Verb+Acquire^DB+Verb+Caus^DB+Verb+Pass+Pos^DB+Noun+PastPart+A3sg+P3sg+Acc
karşı+Postp+PCDat <=> karşı+Adj
şu+Det+Noun+A3sg+Pnon+Nom <=> şu+Det
sık+Verb^DB+Verb+Pass+Pos+Aor+A3pl <=> sıkıl+Verb+Pos+Aor+A3pl
yeniyet+Noun+A3pl+Pnon+Loc <=> yeniyetme+Noun+A3pl+Pnon+Dat
kesen+Noun+A3sg+Pnon+Nom <=> kes+Verb+Pos^DB+Adj+PresPart
karmakarıklışım+Noun+A3sg+Pnon+Nom <=> karmakarışık+Adj^DB+Verb+Zero+Past+A3sg
okullu+Adj+Noun+A3sg+Pnon+Nom <=> okullu+Noun+A3sg+Pnon+Nom
takıl+Verb+Pos+Fut+A3sg <=> tak+Verb^DB+Verb+Pass+Pos+Fut+A3sg
noktala+Verb+Pos+Opt+A3pl <=> noktalıyalım+Noun+A3sg+Pnon+Nom
söyle+Verb+Pos^DB+Noun+PastPart+A3sg+P2pl+Loc <=> söyle+Verb+Pos^DB+Noun+PastPart+A3pl+P2pl+Dat
as+Verb+N

bulun+Verb+Pos^DB+Adj+PastPart+P3sg <=> bulun+Verb+Pos^DB+Noun+PastPart+A3sg+P3sg+Nom
11+Num+Ord+Noun+A3sg+Pnon+Nom <=> 11+Num+Ord
hayat'ın+Noun+A3sg+Pnon+Nom <=> Hayat+Noun+Prop+A3sg+Pnon+Gen
iskenderiye+Noun+A3sg+Pnon+Nom <=> İskenderiye+Noun+Prop+A3sg+Pnon+Nom
gönder+Verb^DB+Verb+Pass+Pos^DB+Adj+PastPart+P3sg <=> gönder+Verb^DB+Verb+Pass+Pos^DB+Noun+PastPart+A3sg+P3sg+Nom
fıb+Noun+A3sg+Pnon+Nom <=> Fbı+Noun+Prop+A3sg+Pnon+Nom
kat+Noun+A3pl+P3pl+Nom <=> kat+Noun+A3pl+P3sg+Nom
mektup+Noun+A3sg+P3sg+Nom <=> mektup+Noun+A3sg+Pnon+Acc
Tgrt+Noun+Prop+A3sg+Pnon+Gen <=> tgrt'nin+Noun+A3sg+Pnon+Nom
sayfa+Noun+A3pl+P3sg+Loc <=> sayfa+Noun+A3pl+P3pl+Loc
Kansas+Noun+Prop+A3sg+Pnon+Loc^DB+Adj+Rel <=> kansas'taki+Noun+A3sg+Pnon+Nom
Kansas+Noun+Prop+A3sg+Pnon+Loc^DB+Adj+Rel <=> kansas'taki+Noun+A3sg+Pnon+Nom
salma+Noun+A3sg+Pnon+Nom <=> sal+Verb+Pos^DB+Noun+Inf2+A3sg+Pnon+Nom
eylem+Noun+A3sg+Pnon+Acc <=> eylem+Noun+A3sg+P3sg+Nom
Fıb+Noun+Prop+A3sg+Pnon+Nom <=> Fbı+Noun+Prop+A3sg+Pnon+Nom
ilgi+Noun

ancak+Conj+Noun+A3sg+Pnon+Nom <=> ancak+Conj
nasıl+Adj <=> nasıl+Adverb
nasıl+Adj <=> nasıl+Adverb
uğra+Verb+Pos^DB+Adverb+AfterDoingSo+Noun+A3sg+Pnon+Dat <=> uğra+Verb+Pos^DB+Adverb+AfterDoingSo
uza+Verb+Pos+Past+Cond+A3sg <=> uzadıya+Adverb
kaçınılmaz+Adj^DB+Verb+Zero+Pres+A1sg <=> kaçınılmaz+Adj^DB+Verb+Zero+Past+A3sg
hastalık+Noun+A3pl+Pnon+Acc <=> hastalık+Noun+A3pl+P3sg+Nom
gözard+Noun+A3sg+Pnon+Acc <=> gözard+Noun+A3sg+P3sg+Nom
et+Verb+Pos^DB+Noun+Inf2+A3sg+Pnon+Nom^DB+Noun+Agt+A3sg+Pnon+Gen <=> et+Verb+Pos^DB+Adverb+WithoutHavingDoneSo
al+Verb+Pos^DB+Noun+Inf2+A3sg+Pnon+Nom^DB+Noun+Agt+A3sg+Pnon+Gen <=> al+Verb+Pos^DB+Adverb+WithoutHavingDoneSo
Ameliyat+Noun+Prop+A3sg+Pnon+Nom <=> ameliyat+Noun+A3sg+Pnon+Nom
organ+Noun+A3pl+P3sg+Nom <=> organ+Noun+A3pl+Pnon+Acc
ol+Verb+Pos^DB+Adverb+AfterDoingSo+Noun+A3sg+Pnon+Dat <=> ol+Verb+Pos^DB+Adverb+AfterDoingSo
ağın+Noun+A3sg+P3sg+Acc <=> ağının+Noun+A3sg+P3sg+Nom
boğ+Verb+Pos+Fut+A3sg <=> boğ+Verb+Pos^DB+Adj+FutPart+Pnon
bu+Det+Noun+A3

0.9201827086080241